In [2]:
import pandas as pd
import torch
from collections import Counter
from sklearn.preprocessing import LabelEncoder

In [3]:
data = pd.read_csv("data/final_data.csv", encoding='utf-8-sig')
data.head()

,번호,약품일반성분명코드(ATC코드),질병명
0,1,['R03A'],천식
1,2,['C07AG02'],고혈압
2,3,['J01D'],세균감염
3,4,"['S', 'S']",non
4,5,['M01AE'],관절염


In [4]:
#질병명이 non인 경우 -> 결측치이므로 제거
data = data[data['질병명']!='non']

#번호 열은 의미없는 값 -> 드랍
data = data.drop(columns=['번호'])

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173055 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   약품일반성분명코드(ATC코드)  173055 non-null  object
 1   질병명               173055 non-null  object
dtypes: object(2)
memory usage: 4.0+ MB


In [27]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

In [28]:
# ATC 코드 vocabulary 생성
code_counter = Counter(code 
                       for _, row in data.iterrows()
                       for code in eval(row['약품일반성분명코드(ATC코드)']))

voca = {code: idx+1 for idx, code in enumerate(code_counter)} #vocabulary 생성

In [21]:
# 라벨 인코딩
label_encoder = LabelEncoder()
data['질병명'] = label_encoder.fit_transform(data['질병명'])
# print(label_encoder.classes_)
print(label_encoder.classes_)

['ADHD' '고지혈증' '고혈압' '관절염' '기침' '녹내장' '당뇨병' '대사질환' '말초혈관질환' '바이러스감염'
 '배뇨장애' '변비' '복합감염' '불면증' '불안장애' '비염' '세균감염' '심부전' '안과감염' '안질환' '역류성식도염'
 '영양결핍' '우울증' '위염' '정신질환' '진균감염' '천식' '통증' '통풍' '피부감염' '피부염' '해독' '혈전예방'
 '호흡기감염' '호흡기질환']


In [26]:
import torch
from torch.utils.data import Dataset, DataLoader

# 파이토치 데이터셋 정의
class PrescriptionDataset(Dataset):
    def __init__(self, df, voca, label_encoder, max_len=5):
        self.voca = voca
        self.label_encoder = label_encoder
        self.max_len = max_len
        self.samples = []

        for _, row in df.iterrows():
            token_ids = [voca.get(code, 0) for code in row['약품일반성분명코드(ATC코드)']]
            token_ids = token_ids[:max_len] + [0] * (max_len - len(token_ids))
            label_id = row['질병명']
            self.samples.append((torch.tensor(token_ids), torch.tensor(label_id)))

    def __len__(self): # len() 호출 시
        return len(self.samples)

    def __getitem__(self, idx): # 인덱스[] 호출 시
        return self.samples[idx]

In [24]:
import torch.nn as nn

# 트랜스포머 분류 모델 정의
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, nhead, num_layers, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 1, embed_dim, padding_idx=0)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=nhead)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):  # x: [batch, seq_len]
        x = self.embedding(x)           # [batch, seq_len, embed_dim]
        x = x.permute(1, 0, 2)          # [seq_len, batch, embed_dim]
        out = self.encoder(x)           # [seq_len, batch, embed_dim]
        out = out.mean(dim=0)           # mean pooling
        return self.fc(out)             # [batch, num_classes]

In [29]:
train_dataset = PrescriptionDataset(train_df, voca, label_encoder)
test_dataset = PrescriptionDataset(test_df, voca, label_encoder)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)

In [30]:
model = TransformerClassifier(
    vocab_size=len(voca),
    embed_dim=32,
    nhead=4,
    num_layers=2,
    num_classes=len(label_encoder.classes_)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    model.train()
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(x_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


c:\Users\moonp\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Loss: 157051.7238
Epoch 2, Loss: 156476.3330
Epoch 3, Loss: 156367.6575
Epoch 4, Loss: 156319.2974
Epoch 5, Loss: 156298.4047


KeyboardInterrupt: 